# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Install and import libraries

In [ ]:
!pip install pyarrow
!pip install pyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.8/329.8 KB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyodbc
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC # "Support vector classifier"  
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import preprocessing
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
pd.options.display.max_rows = None


#Connect with SQL server

In [ ]:
report_driver = '{SQL Server}'
report_server = '172.21.105.125'
report_database = 'Pocket'
report_uid = 'python_user'
report_pwd = 'cnWVDSYVmci5nLyN'

In [ ]:
all_users = pd.DataFrame()
products=pd.DataFrame()

#Data of users and products 

In [ ]:
all_users_query="""select * from [dbo].[Newly Created User Report] where state='active'"""

In [ ]:
product_query="""select v.[user account id],replace(v.[merchant name],'''','') as [merchant name],v.[reward id],
replace(v.[reward name],'''','') as [reward name],v.[voucher id],v.[last engagement status],
convert(date,v.[issued date],20) as [issue date],convert(date,v.[redemption date],20) as [redemption date],
convert(date,v.[expiration date],20) as [expiration date],v.[reward type],v.[redeemed by merchant name],
v.[redeemed by merchant user name],v.[charged points]
from [Voucher Engagement - Breakdown] as v"""

In [ ]:
def db_connect_users_products():
    global all_users,products
    
    conn = pyodbc.connect(driver=report_driver,
                        server=report_server,
                        database=report_database,
                        uid=report_uid,
                        pwd=report_pwd,
                        autocomit=True)
    
    all_users = pd.read_sql(all_users_query,conn) 
    products = pd.read_sql(product_query,conn)    


    conn.close()

In [ ]:
db_connect_users_products()


NameError: ignored

In [ ]:
all_users.columns

In [ ]:
products.columns

In [ ]:
products.to_csv("C:/Users/shineminnkha/Documents/Pocket data/products.csv") # export to csv file on local machine(product data)


#Data preprocessing

In [ ]:
users=pd.read_csv("/content/drive/MyDrive/AI/Yoma/Yoma budget prediction/final_df.csv")

In [ ]:
products=pd.read_csv("/content/drive/MyDrive/AI/Yoma/Recommendation system/products.csv")

In [ ]:
users.columns

Index(['Unnamed: 0', 'user account', 'customer name_x', 'mobile phone_x',
       'birthday', 'perx registration date', 'state', 'email id',
       'registration_source', 'registerd_by_merchant', 'address1', 'city',
       'home', 'gender', 'nationality', 'valid_phone', 'campaign id',
       'customer name_y', 'mobile phone_y', 'merchant name', 'campaign name_x',
       'participated_campaign_frequency', 'campaigns_point_earned',
       'stamp_vouchers', 'stamp_expired_vouchers', 'game_vouchers',
       'game_expired_vouchers', 'is participated?', 'campaign name_y',
       'total_campaign_participated', 'total_sales_point'],
      dtype='object')

In [ ]:
products.columns

Index(['Unnamed: 0', 'user account id', 'merchant name', 'reward id',
       'reward name', 'voucher id', 'last engagement status', 'issue date',
       'redemption date', 'expiration date', 'reward type',
       'redeemed by merchant name', 'redeemed by merchant user name',
       'charged points'],
      dtype='object')

In [ ]:
products['merchant name'].unique()

array(['KFC', 'Pocket', 'YKKO', 'heal by Pun Hlaing', 'Yoma Family Day',
       'Star City', 'Pun Hlaing Hospital', 'Demo Merchant', 'TNY',
       'Horizon Restaurant', 'Balloons Over Bagan (YGN)', 'The Bleachers'],
      dtype=object)

In [ ]:
products['user account id'] = products['user account id'].astype(str)
users['user account'] = users['user account'].astype(str)
product_user=pd.merge(users,products,how='left',left_on=['user account'],right_on=['user account id'])

In [ ]:
product_user.columns

Index(['Unnamed: 0_x', 'user account', 'customer name_x', 'mobile phone_x',
       'birthday', 'perx registration date', 'state', 'email id',
       'registration_source', 'registerd_by_merchant', 'address1', 'city',
       'home', 'gender', 'nationality', 'valid_phone', 'campaign id',
       'customer name_y', 'mobile phone_y', 'merchant name_x',
       'campaign name_x', 'participated_campaign_frequency',
       'campaigns_point_earned', 'stamp_vouchers', 'stamp_expired_vouchers',
       'game_vouchers', 'game_expired_vouchers', 'is participated?',
       'campaign name_y', 'total_campaign_participated', 'total_sales_point',
       'Unnamed: 0_y', 'user account id', 'merchant name_y', 'reward id',
       'reward name', 'voucher id', 'last engagement status', 'issue date',
       'redemption date', 'expiration date', 'reward type',
       'redeemed by merchant name', 'redeemed by merchant user name',
       'charged points'],
      dtype='object')

#Filter purchased or redeemed products

In [ ]:
used_product=product_user[product_user['last engagement status']=='redeemed']

In [ ]:
product= used_product[['user account id','reward name', 'participated_campaign_frequency', 'campaigns_point_earned', 
         'stamp_vouchers', 'game_vouchers', 'total_campaign_participated', 'total_sales_point']]

In [ ]:
len(product.columns)

8

In [ ]:
len(product.index)

1621080

In [ ]:
len(product['reward name'].unique())

60

In [ ]:
# Drop duplicates of the 'Name' column
filtered_product = product.drop_duplicates(subset=['reward name'])



In [ ]:
len(filtered_product.index)

60

In [ ]:
filtered_product.to_csv("/content/drive/MyDrive/AI/Yoma/Recommendation system/filtered_product.csv") # export to csv file on local machine(product data)

In [ ]:
# product[product['reward name'] == 'Get 2 000 MMK Reward Voucher']

#Combine unique filters with the origin

In [ ]:
unique_products=pd.read_csv('/content/drive/MyDrive/AI/Yoma/Recommendation system/filtered_product.csv')

In [ ]:
unique_products.columns

Index(['Unnamed: 0', 'user account id', 'reward name', 'Tags',
       'participated_campaign_frequency', 'campaigns_point_earned',
       'stamp_vouchers', 'game_vouchers', 'total_campaign_participated',
       'total_sales_point'],
      dtype='object')

In [ ]:
len(product_user.index)

5629200

In [ ]:
pocket = pd.merge(product_user,unique_products, on='reward name')


In [ ]:
pocket_data = pocket.drop(['participated_campaign_frequency_x','campaigns_point_earned_x','stamp_vouchers_x',
                           'game_vouchers_x'], axis=1)


In [ ]:
len(pocket_data.index)

3809460

#Content-based recommendation 
##Recommendation based on user information and purchased multiple items

In [ ]:
pocket_data.isnull().sum()

Unnamed: 0_x                               0
user account                               0
customer name_x                         1380
mobile phone_x                          1380
birthday                                2340
perx registration date                  1380
state                                   1380
email id                                2340
registration_source                     1380
registerd_by_merchant                3809460
address1                             3809460
city                                 3809460
home                                 3809460
gender                                  1380
nationality                          3809460
valid_phone                             1380
campaign id                                0
customer name_y                      3315211
mobile phone_y                       3315211
merchant name_x                      3315211
campaign name_x                      3315211
participated_campaign_frequency_x    3315211
campaigns_

In [ ]:
# reward_name=['FREE Coke from KFC နဲ့တူတူ SHAKE မယ်!', 'Get Chicken Reward']
# target_data = user_data[user_data['reward name'].isin(reward_name)] #working
# all_data = product[~product['reward name'].isin(reward_name)]


In [ ]:
result = pocket_data.loc[pocket_data['user account'] == '10016499715-1669597174','reward name'] #take id of a particular user

In [ ]:
result_tag = pocket_data.loc[pocket_data['user account'] == '10016499715-1669597174','Tags'] #take id of a particular user

In [ ]:
def recommend_products_multiple(user_id, reward_name, num_recommendations=10):
    # Load the data and filter for the specified user and reward name
    user_data = pocket_data[pocket_data['user account'] == user_id]
    # target_data = user_data[user_data['reward name'] == reward_name]
    target_data = user_data[user_data['reward name'].isin(reward_name)] 
    # Extract the relevant features
    feature_cols = ['participated_campaign_frequency', 'campaigns_point_earned',
                    'stamp_vouchers', 'game_vouchers','Tags'
                    'total_campaign_participated', 'total_sales_point']
    target_features = target_data[feature_cols]
    # Compute cosine similarity between the target pocket_data and all other pocket_datas
    # all_data = pocket_data[pocket_data['reward name'] != reward_name]
    all_data = pocket_data[~pocket_data['reward name'].isin(reward_name)]
    all_features = all_data[feature_cols]
    similarities = cosine_similarity(target_features, all_features)

    # Get the indices of the top similar pocket_datas
    similar_indices = similarities.argsort()[0][::-1][:num_recommendations]

    # Get the reward names of the recommended pocket_datas
    recommended_rewards = all_data.iloc[similar_indices]['reward name'].tolist()

    return recommended_rewards

In [3]:
# result.unique() #user's actual bought items

In [2]:
# result_tag.unique() #user's actual bought items

In [1]:
# #Recommendation based on multiple items
# recommendations = recommend_products_multiple('10016499715-1669597174', result.unique())
# print(list(set(recommendations)))

#Content-based recommendation 
##Recommendation based on user information and purchased single item

In [ ]:
len(product['reward name'].index) #60 #1621080

1621080

In [ ]:
product['reward name'].unique()

array(['FREE Coke from KFC နဲ့တူတူ SHAKE မယ်!', 'Get Chicken Reward',
       'သံဗူးဖျော်ရည်ဘောင်ချာ', 'Get 3 000 MMK Reward Voucher',
       'KFC Spicy Fried Chicken', 'Get 2 000 MMK Reward Voucher',
       'Get 1 000 MMK Reward Voucher',
       'FREE Coke from Bucket ကြီးခွဲမယ်!', '500 Kyats Worth Voucher',
       '1 000 Kyats Worth Voucher', '1 500 Kyats Worth Voucher',
       '2 000 Kyats Worth Voucher', 'Youve got FREE Large Fries!',
       '12.12 မှာ 1200 coins ဖြင့် ကြက်ကြော်တစ်တုံးယူပါ!',
       '3 000 Kyats Worth Voucher', 'ကျောက်ကျောဖျော်ရည်ဘောင်ချာ',
       '(ကြာဆံ)ကြက်ကြေးအိုးဆီချက်ပွဲသေးဘောင်ချာ', 'အာလူးထောင်းဘောင်ချာ',
       '(ကြာဆံ)ဝက်ကြေးအိုးပွဲသေးဘောင်ချာ', 'YKKO Zee & Black Jelly Drink',
       'Get 2pc Chicken Reward', 'Get Fries Reward',
       '(ကြာဆံ)ဝက်ကြေးအိုးဆီချက်ပွဲသေးဘောင်ချာ',
       'ဝက်ဖတ်ထုတ်ကြော်ဘောင်ချာ', '(ကြာဆံ)ကြက်ကြေးအိုးပွဲသေးဘောင်ချာ',
       'Grilled Chicken Meatballs', 'Get Coke Reward',
       'ကြက်ဖက်ထုတ်ကြော်ဘောင်ချာ', 'Youve got FREE Coke!'

In [ ]:
def recommend_products_single(user_id, reward_name, num_recommendations=10):
    # Load the data and filter for the specified user and reward name
    user_data = product[product['user account id'] == user_id]
    target_data = user_data[user_data['reward name'] == reward_name]
    # Extract the relevant features
    feature_cols = ['participated_campaign_frequency', 'campaigns_point_earned',
                    'stamp_vouchers', 'game_vouchers',
                    'total_campaign_participated', 'total_sales_point']
    target_features = target_data[feature_cols]
    # Compute cosine similarity between the target product and all other products
    all_data = product[product['reward name'] != reward_name]
    all_features = all_data[feature_cols]
    similarities = cosine_similarity(target_features, all_features)

    # Get the indices of the top similar products
    similar_indices = similarities.argsort()[0][::-1][:num_recommendations]

    # Get the reward names of the recommended products
    recommended_rewards = all_data.iloc[similar_indices]['reward name'].tolist()
    return recommended_rewards


In [ ]:
#Real purchased products by the user
result.unique()

array(['Get Chicken Reward', 'Get 2 000 MMK Reward Voucher',
       'Get 1 000 MMK Reward Voucher'], dtype=object)

In [ ]:
#Recommendation based on one single item
recommendations = recommend_products_single('10016499715-1669597174','Get Chicken Reward')
print(list(set(recommendations)))


['Get 2pc Chicken Reward', 'FREE Coke from KFC နဲ့တူတူ SHAKE မယ်!', '1 500 Kyats Worth Voucher', 'Get Fries Reward', 'Get Coke Reward', 'KFC Spicy Fried Chicken', '2 000 Kyats Worth Voucher']


In [ ]:
# #based on multiple products
# ['သံဗူးဖျော်ရည်ဘောင်ချာ', 'FREE Coke from KFC နဲ့တူတူ SHAKE မယ်!', 'Get Coke Reward',
#  'Get 2pc Chicken Reward', 'Get Fries Reward']

# #based on single product
# ['FREE Coke from KFC နဲ့တူတူ SHAKE မယ်!', 
# 'Get Coke Reward','Get 2pc Chicken Reward','Get Fries Reward',
#  '2 000 Kyats Worth Voucher', '1 500 Kyats Worth Voucher', 'KFC Spicy Fried Chicken']


['FREE Coke from KFC နဲ့တူတူ SHAKE မယ်!',
 'Get Coke Reward',
 'Get 2pc Chicken Reward',
 'Get Fries Reward',
 '2 000 Kyats Worth Voucher',
 '1 500 Kyats Worth Voucher',
 'KFC Spicy Fried Chicken']

#Content-based recommendation 
##Recommendation based on product information and purchased single item



In [ ]:
# Define the features to be used for recommendation
features = ['merchant name', 'reward name']

# Create a new dataframe with only the relevant features
df = products[features]

# Clean the text in the 'reward name' column
# df['reward name'] = df['reward name'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex=True)

# Group the dataframe by 'merchant name' and concatenate the 'reward name' column values for each group
grouped_df = df.groupby('merchant name')['reward name'].apply(lambda x: ','.join(x)).reset_index()

# Create a count vectorizer to transform the text into vectors
vectorizer = CountVectorizer(stop_words='english')

# Transform the text into vectors using the count vectorizer
vectorized_text = vectorizer.fit_transform(grouped_df['reward name'])

# Calculate the cosine similarity matrix between the vectors
cosine_sim_matrix = cosine_similarity(vectorized_text)

# Define the function to recommend products based on the selected product's 'reward name'
def recommend_products_based(user_id, reward_name, top_n=1):
    # Get the merchant name of the selected product
    selected_merchant = df.loc[df['reward name'] == reward_name, 'merchant name'].values[0]

    # Get the index of the selected merchant in the cosine similarity matrix
    merchant_idx = grouped_df[grouped_df['merchant name'] == selected_merchant].index[0]

    # Get the cosine similarity scores for the selected merchant
    merchant_scores = cosine_sim_matrix[merchant_idx]

    # Get the indices of the top N products based on similarity scores
    top_n_indices = merchant_scores.argsort()[-top_n-1:-1][::-1]

    # Get the recommended products
    recommended_products = []
    for idx in top_n_indices:
        recommended_products.append(grouped_df.loc[idx, 'reward name'])
    return list(set(recommended_products[0].split(",")))
# list(set(recommended_products[1].split(",")))

# list(set(recommended_products))[0].split(",")

In [ ]:
# recommendation based on merchants
recommend_products_based('16597600954-1670587374','Get 2 000 MMK Reward Voucher')

['Sign In 1000 Voucher',
 'YKKO Zee & Black Jelly Drink',
 'Sign In 2000 Voucher',
 'KFC Spicy Fried Chicken',
 'Challenger Prize (Coins ပေါင်း ၅ ၀၀၀)']

In [ ]:
result = product.loc[product['user account id'] == '1759529049-1670809673','reward name']

In [ ]:
result.unique()

array(['Get Chicken Reward', 'Get 1 000 MMK Reward Voucher',
       'Get 2 000 MMK Reward Voucher',
       '12.12 မှာ 1200 coins ဖြင့် ကြက်ကြော်တစ်တုံးယူပါ!'], dtype=object)

#Merchant recommendation(based on product, merchant is recommended)

In [ ]:
# Filter the dataframe by the input reward name
reward_name = '12.12 မှာ 1200 coins ဖြင့် ကြက်ကြော်တစ်တုံးယူပါ!'
df_filtered = products[products['reward name'] == reward_name]

# Get the unique merchant names for the filtered dataframe
merchant_names = df_filtered['merchant name'].unique()

# Create a new dataframe with one row for each unique merchant name
df_merchants = pd.DataFrame({'merchant name': merchant_names})

# Build a corpus of merchant names for vectorization
merchant_corpus = df_merchants['merchant name'].tolist()

# Vectorize the merchant corpus
vectorizer = CountVectorizer()
merchant_matrix = vectorizer.fit_transform(merchant_corpus)

# Calculate the cosine similarity matrix for the merchant matrix
merchant_similarity = cosine_similarity(merchant_matrix)

# Find the most similar merchant names to each merchant name
recommendations = {}
for i, merchant in enumerate(merchant_names):
    similarity_scores = list(merchant_similarity[i])
    most_similar_indices = sorted(range(len(similarity_scores)), key=lambda k: similarity_scores[k], reverse=True)[1:6]
    most_similar_merchants = [merchant_names[idx] for idx in most_similar_indices]
    recommendations[merchant] = most_similar_merchants

# Print the recommendations for the input reward name
print(f"Recommendations for reward name '{reward_name}':")
for merchant, similar_merchants in recommendations.items():
    print(f"For merchant '{merchant}', we recommend: {', '.join(similar_merchants)}")


Recommendations for reward name '12.12 မှာ 1200 coins ဖြင့် ကြက်ကြော်တစ်တုံးယူပါ!':
For merchant 'KFC', we recommend: 


In [ ]:
['Youve got FREE Coke!', 'Get Fries Reward', 'Get Chicken Reward',
       'Get 3 000 MMK Reward Voucher',
       '20% discount coupon (medical checkup package)',
       'FREE Coke from KFC နဲ့တူတူ SHAKE မယ်!',
       '12.12 မှာ 1200 coins ဖြင့် ကြက်ကြော်တစ်တုံးယူပါ!']